Importing Data

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [37]:
#Import all excel files
dimension_tables_df = pd.read_excel('/content/DimensionTables.xlsx')
sales_2017_df = pd.read_excel('/content/Sales 2017.xlsx')
sales_2018_df = pd.read_excel('/content/Sales 2018.xlsx')
sales_2019_df = pd.read_excel('/content/Sales 2019.xlsx')
targets_df = pd.read_excel('/content/Targets.xlsx', skiprows=4)

In [38]:
# Read individual dimension tables
file_path = 'DimensionTables.xlxs'
customer_df = pd.read_excel('/content/DimensionTables.xlsx', sheet_name='Customer')
product_df = pd.read_excel('/content/DimensionTables.xlsx', sheet_name='Product')
product_group_df = pd.read_excel('/content/DimensionTables.xlsx', sheet_name='ProductGroup')
sales_person_df = pd.read_excel('/content/DimensionTables.xlsx', sheet_name='SalesPerson')
dates_df = pd.read_excel('/content/DimensionTables.xlsx', sheet_name='Dates')

Data Merging

In [39]:
# Since sales_2017, sales_2018, and sales_2019 have the same structure, we will concatenate sales dataframes together
sales_df = pd.concat([sales_2017_df, sales_2018_df, sales_2019_df], ignore_index=True)

In [40]:
sales_df = pd.merge(sales_df, dates_df, left_on='Issue Date', right_on='Dates', how='left')
sales_df = pd.merge(sales_df, customer_df, on='Customer ID', how='right')
sales_df = pd.merge(sales_df, sales_person_df, on='SalesPerson ID', how='left')
sales_df = pd.merge(sales_df, product_df, on='Product ID', how='left')
sales_df = pd.merge(sales_df, product_group_df, on='Group ID', how='left')
merged_df = sales_df

In [41]:
merged_df.info()
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260235 entries, 0 to 260234
Data columns (total 29 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Issue Date        260096 non-null  datetime64[ns]
 1   Due Date          260096 non-null  datetime64[ns]
 2   Invoice Number    260096 non-null  float64       
 3   Customer ID       260235 non-null  int64         
 4   SalesPerson ID    260096 non-null  float64       
 5   Product ID        260096 non-null  float64       
 6   Qty Itens         260096 non-null  float64       
 7   Unit Price        260096 non-null  float64       
 8   Net Weight        260096 non-null  float64       
 9   Dates             260096 non-null  datetime64[ns]
 10  Year              260096 non-null  float64       
 11  Month Name        260096 non-null  object        
 12  Month             260096 non-null  float64       
 13  Day               260096 non-null  float64       
 14  Comp

,Issue Date,Due Date,Invoice Number,Customer ID,SalesPerson ID,Product ID,Qty Itens,Unit Price,Net Weight,Dates,...,SalesPerson,Supervisor ID,Supervisor,Manager ID,Manager,Team,Product,Group ID,Group,Category
0,2017-11-25,2018-01-03,1823678.0,1339104,669.0,2105.0,2.0,85.67,0.63,2017-11-25,...,Ayaana Mackenzie,15.0,Jonathon Childs,1.0,Ronnie Daly,Retail,Product 2105,213.0,Sugarcane Liquor,Drinks
1,2017-11-25,2018-01-03,1823678.0,1339104,669.0,2103.0,2.0,81.60,1.14,2017-11-25,...,Ayaana Mackenzie,15.0,Jonathon Childs,1.0,Ronnie Daly,Retail,Product 2103,213.0,Sugarcane Liquor,Drinks
2,2017-12-21,2018-01-27,1838494.0,1339104,669.0,1376.0,4.0,10.64,0.84,2017-12-21,...,Ayaana Mackenzie,15.0,Jonathon Childs,1.0,Ronnie Daly,Retail,Product 1376,210.0,Red Sweet Wine,Drinks
3,2017-12-21,2018-01-27,1838494.0,1339104,669.0,1377.0,4.0,10.64,0.96,2017-12-21,...,Ayaana Mackenzie,15.0,Jonathon Childs,1.0,Ronnie Daly,Retail,Product 1377,210.0,Red Sweet Wine,Drinks
4,2017-12-21,2018-01-27,1838494.0,1339104,669.0,2105.0,2.0,85.67,0.63,2017-12-21,...,Ayaana Mackenzie,15.0,Jonathon Childs,1.0,Ronnie Daly,Retail,Product 2105,213.0,Sugarcane Liquor,Drinks


Data Cleaning

In [42]:
# Rename specific columns
merged_df = merged_df.rename(columns={'Year': 'Order Year', 'Month': 'Order Month'})
merged_df = merged_df.rename(columns={'Qty Itens': 'Quantity'})

# Rename values in the Status column
merged_df['Status'] = merged_df['Status'].replace({'A': 'Active', 'I': 'Inactive'})

# Verify the changes
print(merged_df['Status'].value_counts())
merged_df.head()

Status
Active      251763
Inactive      8472
Name: count, dtype: int64


,Issue Date,Due Date,Invoice Number,Customer ID,SalesPerson ID,Product ID,Quantity,Unit Price,Net Weight,Dates,...,SalesPerson,Supervisor ID,Supervisor,Manager ID,Manager,Team,Product,Group ID,Group,Category
0,2017-11-25,2018-01-03,1823678.0,1339104,669.0,2105.0,2.0,85.67,0.63,2017-11-25,...,Ayaana Mackenzie,15.0,Jonathon Childs,1.0,Ronnie Daly,Retail,Product 2105,213.0,Sugarcane Liquor,Drinks
1,2017-11-25,2018-01-03,1823678.0,1339104,669.0,2103.0,2.0,81.60,1.14,2017-11-25,...,Ayaana Mackenzie,15.0,Jonathon Childs,1.0,Ronnie Daly,Retail,Product 2103,213.0,Sugarcane Liquor,Drinks
2,2017-12-21,2018-01-27,1838494.0,1339104,669.0,1376.0,4.0,10.64,0.84,2017-12-21,...,Ayaana Mackenzie,15.0,Jonathon Childs,1.0,Ronnie Daly,Retail,Product 1376,210.0,Red Sweet Wine,Drinks
3,2017-12-21,2018-01-27,1838494.0,1339104,669.0,1377.0,4.0,10.64,0.96,2017-12-21,...,Ayaana Mackenzie,15.0,Jonathon Childs,1.0,Ronnie Daly,Retail,Product 1377,210.0,Red Sweet Wine,Drinks
4,2017-12-21,2018-01-27,1838494.0,1339104,669.0,2105.0,2.0,85.67,0.63,2017-12-21,...,Ayaana Mackenzie,15.0,Jonathon Childs,1.0,Ronnie Daly,Retail,Product 2105,213.0,Sugarcane Liquor,Drinks


In [43]:
merged_df.drop_duplicates(inplace=True)
merged_df.drop(columns=['Supervisor ID', 'Product', 'Group ID', 'Manager ID',
                        'Dates', 'Day', 'Order Month',
                        'Line of Business'],
               inplace=True)

In [44]:
merged_df.head()

,Issue Date,Due Date,Invoice Number,Customer ID,SalesPerson ID,Product ID,Quantity,Unit Price,Net Weight,Order Year,...,Company Name,Status,City,State,SalesPerson,Supervisor,Manager,Team,Group,Category
0,2017-11-25,2018-01-03,1823678.0,1339104,669.0,2105.0,2.0,85.67,0.63,2017.0,...,Dwarf Acoustics,Active,Winthrop,WA,Ayaana Mackenzie,Jonathon Childs,Ronnie Daly,Retail,Sugarcane Liquor,Drinks
1,2017-11-25,2018-01-03,1823678.0,1339104,669.0,2103.0,2.0,81.60,1.14,2017.0,...,Dwarf Acoustics,Active,Winthrop,WA,Ayaana Mackenzie,Jonathon Childs,Ronnie Daly,Retail,Sugarcane Liquor,Drinks
2,2017-12-21,2018-01-27,1838494.0,1339104,669.0,1376.0,4.0,10.64,0.84,2017.0,...,Dwarf Acoustics,Active,Winthrop,WA,Ayaana Mackenzie,Jonathon Childs,Ronnie Daly,Retail,Red Sweet Wine,Drinks
3,2017-12-21,2018-01-27,1838494.0,1339104,669.0,1377.0,4.0,10.64,0.96,2017.0,...,Dwarf Acoustics,Active,Winthrop,WA,Ayaana Mackenzie,Jonathon Childs,Ronnie Daly,Retail,Red Sweet Wine,Drinks
4,2017-12-21,2018-01-27,1838494.0,1339104,669.0,2105.0,2.0,85.67,0.63,2017.0,...,Dwarf Acoustics,Active,Winthrop,WA,Ayaana Mackenzie,Jonathon Childs,Ronnie Daly,Retail,Sugarcane Liquor,Drinks


In [45]:
# Adding new column
merged_df['Revenue'] = merged_df['Quantity'] * merged_df['Unit Price']

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260235 entries, 0 to 260234
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Issue Date      260096 non-null  datetime64[ns]
 1   Due Date        260096 non-null  datetime64[ns]
 2   Invoice Number  260096 non-null  float64       
 3   Customer ID     260235 non-null  int64         
 4   SalesPerson ID  260096 non-null  float64       
 5   Product ID      260096 non-null  float64       
 6   Quantity        260096 non-null  float64       
 7   Unit Price      260096 non-null  float64       
 8   Net Weight      260096 non-null  float64       
 9   Order Year      260096 non-null  float64       
 10  Month Name      260096 non-null  object        
 11  Company Name    260235 non-null  object        
 12  Status          260235 non-null  object        
 13  City            260235 non-null  object        
 14  State           260235 non-null  obj

In [46]:
# Define the new column order
new_order = ['Issue Date', 'Due Date', 'Invoice Number', 'SalesPerson ID', 'Customer ID',
             'Product ID', 'Quantity', 'Unit Price',
             'Revenue', 'Net Weight', 'Order Year', 'Month Name', 'Company Name', 'Status',
             'City', 'State', 'SalesPerson', 'Supervisor', 'Manager', 'Team', 'Group', 'Category']

# Reorder columns
merged_df = merged_df[new_order]

# Verify the new order
print(merged_df.head())

  Issue Date   Due Date  Invoice Number  SalesPerson ID  Customer ID  \
0 2017-11-25 2018-01-03       1823678.0           669.0      1339104   
1 2017-11-25 2018-01-03       1823678.0           669.0      1339104   
2 2017-12-21 2018-01-27       1838494.0           669.0      1339104   
3 2017-12-21 2018-01-27       1838494.0           669.0      1339104   
4 2017-12-21 2018-01-27       1838494.0           669.0      1339104   

   Product ID  Quantity  Unit Price  Revenue  Net Weight  ...  \
0      2105.0       2.0       85.67   171.34        0.63  ...   
1      2103.0       2.0       81.60   163.20        1.14  ...   
2      1376.0       4.0       10.64    42.56        0.84  ...   
3      1377.0       4.0       10.64    42.56        0.96  ...   
4      2105.0       2.0       85.67   171.34        0.63  ...   

      Company Name  Status      City State       SalesPerson       Supervisor  \
0  Dwarf Acoustics  Active  Winthrop    WA  Ayaana Mackenzie  Jonathon Childs   
1  Dwarf Acous

Dealing with the targets excel sheet

In [47]:
targets_df.head()

,SalesPerson ID,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,2017-07-01 00:00:00,2017-08-01 00:00:00,2017-09-01 00:00:00,...,2018-04-01 00:00:00,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-07-01 00:00:00,2018-08-01 00:00:00,2018-09-01 00:00:00,2018-10-01 00:00:00,2018-11-01 00:00:00,2018-12-01 00:00:00,Total
0,102,140768.312981,124258.753484,169509.931262,143465.231352,98423.627768,133401.861987,109124.992168,221597.228027,101603.974771,...,105568.516310,58743.776233,181830.392098,135931.360716,105977.016031,67048.978875,136594.459203,39370.133088,34296.939141,3.033854e+06
1,125,21525.584613,50778.518843,80286.310022,117769.055030,162043.205771,184889.338359,175629.570395,120097.495562,110607.719934,...,108730.296716,44662.552004,264250.812124,80597.535419,122250.986149,114977.701064,199905.377620,52701.549885,46867.563534,2.708057e+06
2,144,72301.437981,47360.516270,54145.357333,89457.661634,115437.266528,72356.845601,164689.100713,102330.724424,112396.578585,...,66398.357121,99162.551567,68546.309842,73499.767441,168450.039438,115770.331253,101263.109944,16718.894966,32769.773036,2.050896e+06
3,194,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,107117.669385,148042.387145,90060.231633,39364.546546,43339.557259,4.279244e+05
4,196,75253.911490,56094.071733,156709.108284,82723.073158,47426.320283,66398.220086,91840.461612,136680.893152,87425.550099,...,43642.489333,73916.754356,114445.340468,76255.523468,138085.163730,58425.345277,67537.642857,10161.933261,10380.642211,2.037217e+06


In [48]:
new_index = pd.MultiIndex.from_tuples([
    (102, 2017),
    (102, 2018),
    (125, 2017),
    (125, 2018),
    (144, 2017),
    (144, 2018),
    (194, 2017),
    (194, 2018),
    (196, 2017),
    (196, 2018),
    (215, 2017),
    (215, 2018),
    (265, 2017),
    (265, 2018),
    (285, 2017),
    (285, 2018),
    (326, 2017),
    (326, 2018),
    (660, 2017),
    (660, 2018),
    (669, 2017),
    (669, 2018),
], names =['SalesPerson ID', 'Year'])

new_targets_df = pd.DataFrame(index=new_index, columns=targets_df.columns[1:-1])

In [49]:
new_targets_df.head()

2017-01-01 2017-02-01 2017-03-01 2017-04-01 2017-05-01  \
SalesPerson ID Year                                                          
102            2017        NaN        NaN        NaN        NaN        NaN   
               2018        NaN        NaN        NaN        NaN        NaN   
125            2017        NaN        NaN        NaN        NaN        NaN   
               2018        NaN        NaN        NaN        NaN        NaN   
144            2017        NaN        NaN        NaN        NaN        NaN   

                    2017-06-01 2017-07-01 2017-08-01 2017-09-01 2017-10-01  \
SalesPerson ID Year                                                          
102            2017        NaN        NaN        NaN        NaN        NaN   
               2018        NaN        NaN        NaN        NaN        NaN   
125            2017        NaN        NaN        NaN        NaN        NaN   
               2018        NaN        NaN        NaN        NaN        NaN   
144            2017        NaN        NaN        NaN        NaN        NaN   

                     ... 2018-03-01 2018-04-01 2018-05-01 2018-06-01  \
SalesPerson ID Year  ...                                               
102            2017  ...        NaN        NaN        NaN        NaN   
               2018  ...        NaN        NaN        NaN        NaN   
125            2017  ...        NaN        NaN        NaN        NaN   
               2018  ...        NaN        NaN        NaN        NaN   
144            2017  ...        NaN        NaN        NaN        NaN   

                    2018-07-01 2018-08-01 2018-09-01 2018-10-01 2018-11-01  \
SalesPerson ID Year                                                          
102            2017        NaN        NaN        NaN        NaN        NaN   
               2018        NaN        NaN        NaN        NaN        NaN   
125            2017        NaN        NaN        NaN        NaN        NaN   
               2018        NaN        NaN        NaN        NaN        NaN   
144            2017        NaN        NaN        NaN        NaN        NaN   

                    2018-12-01  
SalesPerson ID Year             
102            2017        NaN  
               2018        NaN  
125            2017        NaN  
               2018        NaN  
144            2017        NaN  

[5 rows x 24 columns]

In [50]:
# First, let's extract the monthly data from the original targets sheet
# The first sheet has monthly data from 2017-01 to 2018-12 (24 months)

# Create month columns for the new DataFrame (just 1-12 for monthly data)
month_columns = list(range(1, 13))  # [1, 2, 3, ..., 12]

# Initialize the new DataFrame with NaN values
new_targets_df = pd.DataFrame(index=new_index, columns=month_columns)

# Populate the data by iterating through each SalesPerson ID and year
for (salesperson_id, year) in new_index:
    # Find the row in the original data for this salesperson
    original_row = targets_df[targets_df['SalesPerson ID'] == salesperson_id]

    if not original_row.empty:
        if year == 2017:
            # 2017 data: columns B to M (months 1-12 of 2017)
            monthly_data = original_row.iloc[0, 1:13]  # B to M
            new_targets_df.loc[(salesperson_id, year)] = monthly_data.values
        elif year == 2018:
            # 2018 data: columns N to Y (months 1-12 of 2018)
            monthly_data = original_row.iloc[0, 13:25]  # N to Y
            new_targets_df.loc[(salesperson_id, year)] = monthly_data.values



In [51]:
#Format decimal places to 2
new_targets_df = new_targets_df.round(2)
new_targets_df.head()

1              2              3   \
SalesPerson ID Year                                                
102            2017  140768.312981  124258.753484  169509.931262   
               2018    82260.75849    39925.04843  315935.618731   
125            2017   21525.584613   50778.518843   80286.310022   
               2018   86536.033101  100700.820173     63898.4213   
144            2017   72301.437981    47360.51627   54145.357333   

                                4              5              6   \
SalesPerson ID Year                                                
102            2017  143465.231352   98423.627768  133401.861987   
               2018   105568.51631   58743.776233  181830.392098   
125            2017   117769.05503  162043.205771  184889.338359   
               2018  108730.296716   44662.552004  264250.812124   
144            2017   89457.661634  115437.266528   72356.845601   

                                7              8              9   \
SalesPerson ID Year                                                
102            2017  109124.992168  221597.228027  101603.974771   
               2018  135931.360716  105977.016031   67048.978875   
125            2017  175629.570395  120097.495562  110607.719934   
               2018   80597.535419  122250.986149  114977.701064   
144            2017  164689.100713  102330.724424  112396.578585   

                                10             11             12  
SalesPerson ID Year                                               
102            2017   55255.942471  261910.628582  171050.965613  
               2018  136594.459203   39370.133088   34296.939141  
125            2017  146219.270646   69770.204547  182360.781748  
               2018   199905.37762   52701.549885   46867.563534  
144            2017   106251.30251   68987.311215   78735.894253

In [52]:
# Create a mapping from month numbers to month names
month_names = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April',
    5: 'May', 6: 'June', 7: 'July', 8: 'August',
    9: 'September', 10: 'October', 11: 'November', 12: 'December'
}

# Rename the columns
new_targets_df = new_targets_df.rename(columns=month_names)

new_targets_df.head()

January       February          March  \
SalesPerson ID Year                                                
102            2017  140768.312981  124258.753484  169509.931262   
               2018    82260.75849    39925.04843  315935.618731   
125            2017   21525.584613   50778.518843   80286.310022   
               2018   86536.033101  100700.820173     63898.4213   
144            2017   72301.437981    47360.51627   54145.357333   

                             April            May           June  \
SalesPerson ID Year                                                
102            2017  143465.231352   98423.627768  133401.861987   
               2018   105568.51631   58743.776233  181830.392098   
125            2017   117769.05503  162043.205771  184889.338359   
               2018  108730.296716   44662.552004  264250.812124   
144            2017   89457.661634  115437.266528   72356.845601   

                              July         August      September  \
SalesPerson ID Year                                                
102            2017  109124.992168  221597.228027  101603.974771   
               2018  135931.360716  105977.016031   67048.978875   
125            2017  175629.570395  120097.495562  110607.719934   
               2018   80597.535419  122250.986149  114977.701064   
144            2017  164689.100713  102330.724424  112396.578585   

                           October       November       December  
SalesPerson ID Year                                               
102            2017   55255.942471  261910.628582  171050.965613  
               2018  136594.459203   39370.133088   34296.939141  
125            2017  146219.270646   69770.204547  182360.781748  
               2018   199905.37762   52701.549885   46867.563534  
144            2017   106251.30251   68987.311215   78735.894253

In [53]:
# Much simpler - sums all month columns automatically
new_targets_df['Total Annual Target'] = new_targets_df[['January', 'February', 'March', 'April',
                                                       'May', 'June', 'July', 'August',
                                                       'September', 'October', 'November',
                                                       'December']].sum(axis=1)

In [54]:
new_targets_df.head(23)

January       February          March  \
SalesPerson ID Year                                                
102            2017  140768.312981  124258.753484  169509.931262   
               2018    82260.75849    39925.04843  315935.618731   
125            2017   21525.584613   50778.518843   80286.310022   
               2018   86536.033101  100700.820173     63898.4213   
144            2017   72301.437981    47360.51627   54145.357333   
               2018   46295.097196   51108.112009  126463.216512   
194            2017            0.0            0.0            0.0   
               2018            0.0            0.0            0.0   
196            2017    75253.91149   56094.071733  156709.108284   
               2018    48769.00116   84907.063554   99185.119353   
215            2017   286012.89817  268739.946775  329813.357769   
               2018   90909.046993  190186.542626  302684.413565   
265            2017  122199.726847  147007.556793  127262.128128   
               2018   58168.106256  111390.695978   254887.01681   
285            2017            0.0            0.0            0.0   
               2018    43710.07517   59640.117308   98667.739624   
326            2017            0.0            0.0            0.0   
               2018            0.0            0.0            0.0   
660            2017            0.0            0.0            0.0   
               2018            0.0            0.0            0.0   
669            2017            0.0            0.0            0.0   
               2018   92534.825581  137971.375033   47613.194688   

                             April            May           June  \
SalesPerson ID Year                                                
102            2017  143465.231352   98423.627768  133401.861987   
               2018   105568.51631   58743.776233  181830.392098   
125            2017   117769.05503  162043.205771  184889.338359   
               2018  108730.296716   44662.552004  264250.812124   
144            2017   89457.661634  115437.266528   72356.845601   
               2018   66398.357121   99162.551567   68546.309842   
194            2017            0.0            0.0            0.0   
               2018            0.0            0.0            0.0   
196            2017   82723.073158   47426.320283   66398.220086   
               2018   43642.489333   73916.754356  114445.340468   
215            2017    159143.4362  117572.134697   275378.65914   
               2018  148977.354813  141750.581959  199966.722033   
265            2017  206980.493152  119509.500049  192623.286112   
               2018   71697.097329  151967.912128  280318.281792   
285            2017            0.0            0.0   39276.029666   
               2018   62042.369773    61236.69049  223984.143878   
326            2017            0.0            0.0            0.0   
               2018            0.0  108976.639574  163632.605958   
660            2017            0.0            0.0            0.0   
               2018            0.0            0.0            0.0   
669            2017            0.0            0.0   68107.204618   
               2018  105228.809015  130951.598498  150327.182692   

                              July         August      September  \
SalesPerson ID Year                                                
102            2017  109124.992168  221597.228027  101603.974771   
               2018  135931.360716  105977.016031   67048.978875   
125            2017  175629.570395  120097.495562  110607.719934   
               2018   80597.535419  122250.986149  114977.701064   
144            2017  164689.100713  102330.724424  112396.578585   
               2018   73499.767441  168450.039438  115770.331253   
194            2017            0.0            0.0            0.0   
               2018            0.0  107117.669385  148042.387145   
196            2017   91840.461612  136680.893152   87425.550099   
             

Exporting the cleaned tables

In [60]:
from google.colab import files

# Export as Excel
merged_df.to_excel('cleaned_sales.xlsx')
files.download('cleaned_sales.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
from google.colab import files

# Export as Excel
new_targets_df.to_excel('cleaned_targets.xlsx')
files.download('cleaned_targets.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>